# Setup

In [ ]:
# === ENVIRONMENT SETUP ===
!pip install -U langchain langchain-core langchain-community langchain-openai
!pip install -q openai tiktoken sentence-transformers faiss-cpu unstructured pymupdf pdf2image pytesseract pillow json5
!apt-get install -y poppler-utils tesseract-ocr
!pip install ipywidgets
!pip install -q docling pymupdf
!pip install -q anthropic
!pip install -q google-generativeai


# === CORE LIBRARIES ===
import os
import re
import json
import json5
import time
import zipfile
import pandas as pd
from pathlib import Path
from IPython.display import FileLink

# === COLAB UTILITIES ===
from google.colab import files

# === PDF + TEXT PARSING ===
from tqdm import tqdm
from docling.document_converter import DocumentConverter
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import PdfFormatOption, InputFormat
from docling.backend.docling_parse_v2_backend import DoclingParseV2DocumentBackend

# === LANGCHAIN CORE ===
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

# === LLM PROVIDERS ===
from openai import OpenAI
import anthropic
import google.generativeai as genai


# === GLOBAL STATE (User Session Variables) ===
full_text = ""
ESG_CHUNKS = []
ESG_VECTORSTORE = None
GRI_ID_MAP = {}
GRI_DEPRECATED_IDS = set()
DISCLOSURES = []
MATCH_RESULTS = []
SELECTED_MODELS = []
METADATA_BY_MODEL = {}
EVAL_RESULTS = {}
FIX_RESULTS = {}
SESSION_LOG = []
GLOBAL_STATE_INDEX = None
RESULTS = {
    "matching": [],
    "metadata": {},
    "evaluation": {},
    "fixes": {}
}


# === DEFAULT MODEL PROVIDERS ===
providers = {
    "gpt": {"provider": "openai", "model": "gpt-4.1-nano"},
    "claude": {"provider": "claude", "model": "claude-3-5-haiku-20241022"},
    "gemini": {"provider": "gemini", "model": "gemini-2.0-flash-lite"}
}

EMBED_MODEL = "text-embedding-3-large"


In [ ]:
os.environ["OPENAI_API_KEY"] = "ENTER_YOUR_API_HERE"
os.environ["ANTHROPIC_API_KEY"] = "ENTER_YOUR_API_HERE"
os.environ["GOOGLE_API_KEY"] = "ENTER_YOUR_API_HERE"

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os

def build_global_state_index(docs, rebuild=False):
    FAISS_INDEX_PATH = "faiss_global_index"
    embedding_model = OpenAIEmbeddings()

    if os.path.exists(FAISS_INDEX_PATH) and not rebuild:
        print("✅ Loading saved FAISS index...")
        return FAISS.load_local(FAISS_INDEX_PATH, embedding_model, allow_dangerous_deserialization=True)

    print("📦 Building FAISS index from scratch...")
    index = FAISS.from_documents(docs, embedding_model)
    index.save_local(FAISS_INDEX_PATH)
    return index


**Output files upload for analysis**

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload e.g., all_outputs.zip


In [ ]:
import zipfile
import os

with zipfile.ZipFile("content_directory_backup.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Confirm
print("✅ Extracted folders:", os.listdir())


# ESG Agent Core Functions Definition

In [ ]:
# === ESG Agent Core Function Definitions ===

# ---------------------------------------
# ✅ Utility
# ---------------------------------------

def extract_json_from_text(text, provider="gpt"):
    text = text.strip("` \n")
    if text.lower().startswith("json"):
        text = text[4:].strip()
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        json_str = match.group()
        json_str = json_str.replace("+", "")
        json_str = re.sub(r",\s*}", "}", json_str)
        json_str = re.sub(r",\s*]", "]", json_str)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return json5.loads(json_str)
    raise ValueError("❌ No JSON structure found in text.")

def token_count(prompt: str, provider: str = "openai") -> int:
    """
    Estimates token count based on provider.
    For OpenAI: exact via tiktoken.
    For Claude: 1 token ≈ 3.5 characters.
    For Gemini: 1 token ≈ 0.75 words.
    """
    if provider == "openai":
        try:
            import tiktoken
            enc = tiktoken.encoding_for_model("gpt-4")  # or use gpt-3.5-turbo if needed
            return len(enc.encode(prompt))
        except Exception:
            return len(prompt.split())  # fallback

    elif provider == "claude":
        return int(len(prompt) / 3.5)

    elif provider == "gemini":
        return int(len(prompt.split()) / 0.75)

    return len(prompt.split())  # fallback

def get_token_usage(response, prompt, output, provider):
    """
    Attempts to extract input/output token counts from model response.
    Falls back to estimation if usage info is not present.

    Args:
        response: The raw LLM response object (if available).
        prompt (str): The prompt text sent to the model.
        output (str): The output text received from the model.
        provider (str): One of: "openai", "claude", "gemini".

    Returns:
        dict: {
            "input_tokens": int,
            "output_tokens": int,
            "total_tokens": int
        }
    """
    input_tokens = output_tokens = 0

    # ✅ OpenAI: Accurate
    if provider == "openai" and hasattr(response, "usage") and response.usage:
        input_tokens = response.usage.prompt_tokens
        output_tokens = response.usage.completion_tokens

    # ✅ Claude: Might not have usage
    elif provider == "claude" and hasattr(response, "usage") and response.usage:
        input_tokens = response.usage.input_tokens
        output_tokens = response.usage.output_tokens

    # ⚠️ Gemini or fallback
    else:
        input_tokens = token_count(prompt, provider)
        output_tokens = token_count(output, provider)

    return {
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": input_tokens + output_tokens
    }




# ---------------------------------------
# ✅ Prompt Builders
# ---------------------------------------

def build_metadata_prompt(disclosure, matched_text):
    return f"""
You are an ESG metadata extraction assistant.

Your task is to extract structured ESG metadata from the matched company report excerpt.

=== DISCLOSURE ===
ID: {disclosure.get('id')}
Title: {disclosure.get('title')}
Description: {disclosure.get('description')}

=== MATCHED TEXT ===
{matched_text}

=== OUTPUT INSTRUCTIONS ===
- Include the full title of the disclosure.
- Extract any **quantitative indicators** (numerical KPIs, metrics, values) found in the text.
- Also include a **qualitative summary** that captures policies, initiatives, or narrative context.
- If no quantitative indicators exist, return an empty list.
- If no qualitative content exists, return an empty string.

=== OUTPUT FORMAT ===
{{
  "disclosure_id": "{disclosure.get('id')}",
  "title": "{disclosure.get('title')}",
  "qualitative_summary": "Brief ESG-relevant summary here.",
  "quantitative_indicators": [
    {{
      "name": "metric_name",
      "value": 123.45,
      "unit": "tons",
      "year": 2022
    }}
  ]
}}

- Do NOT return markdown, bullet points, or any explanation.
- Return only clean JSON exactly in the above format.
""".strip()

def build_metadata_eval_prompt(disclosure, metadata):
    return f"""
You are an ESG compliance auditor.

Evaluate whether the company’s ESG disclosure meets the requirement below.

=== DISCLOSURE REQUIREMENT ===
ID: {disclosure.get("id")}
Title: {disclosure.get("title")}
Description: {disclosure.get("description")}

=== COMPANY METADATA ===
{json.dumps(metadata, indent=2)}

== YOUR TASK ==
- Assign one of the following ratings:
  - Fully Compliant (score: 1.0)
  - Partially Compliant (score: 0.5)
  - Non-Compliant (score: 0.0)

- Justify your decision with clear reasoning (e.g. presence/absence of KPIs, vague policies, incomplete scope).

- Format your output in **valid JSON** exactly like this:
{{
  "rating": "Fully Compliant",
  "score": 1.0,
  "justification": "Includes three KPIs with targets and a clear policy summary."
}}

Respond only with clean JSON. No markdown or extra explanation.
""".strip()


def build_fix_suggestion_prompt(disclosure_id, rating, reason, rule_text):
    return f"""
You are an ESG policy analyst.

A disclosure was marked as '{rating}' in an ESG compliance audit.

Disclosure ID: {disclosure_id}
Rule Description:
{rule_text.strip()}

❌ Gap Identified:
{reason.strip()}

Suggest one clear improvement that could help the company align better with this rule. Return plain text only. No markdown or headers.
""".strip()


# ---------------------------------------
# 👨‍💼 Admin Functions (to be run once)
# ---------------------------------------

def load_gri_mapping_with_deprecated(path):
    import pandas as pd
    df = pd.read_excel(path, sheet_name="2. 2016 to 2021 ", header=25, engine="openpyxl")
    df = df.rename(columns={"Unnamed: 2": "Disclosure_2016", "Unnamed: 7": "Disclosure_2021"})
    df["Disclosure_2016_base"] = df["Disclosure_2016"].astype(str).str.extract(r"(\d{3}-\d+)")
    df = df.dropna(subset=["Disclosure_2016_base"])
    mapping, deprecated = {}, []
    for _, row in df.iterrows():
        old_id = row["Disclosure_2016_base"]
        new_id = row["Disclosure_2021"]
        mapping[old_id] = str(new_id).strip() if pd.notna(new_id) and str(new_id).upper() != "N/A" else None
        if mapping[old_id] is None: deprecated.append(old_id)
    for mid in {"412-1", "412-2", "412-3"}:
        if mid not in mapping:
            mapping[mid] = None
            deprecated.append(mid)
    return mapping, set(deprecated)

def admin_load_gri_mapping_and_save(path):
    mapping, deprecated = load_gri_mapping_with_deprecated(path)
    os.makedirs("admin", exist_ok=True)
    with open("admin/gri_mapping.json", "w") as f:
        json.dump({"mapping": mapping, "deprecated": list(deprecated)}, f)
    print(f"✅ GRI mapping saved. Entries: {len(mapping)}, Deprecated: {len(deprecated)}")


# ---------------------------------------
# 👤 User Runtime Functions
# ---------------------------------------

def load_admin_disclosures():
    with open("admin/all_disclosures.json", "r", encoding="utf-8") as f:
        return json.load(f)

def load_gri_mapping():
    with open("admin/gri_mapping.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return data["mapping"], set(data["deprecated"])

def normalize_gri_id(disclosure_id, mapping):
    base_id = re.match(r"(\d{3}-\d+)", disclosure_id.strip())
    base_id = base_id.group(1) if base_id else disclosure_id.strip()
    return mapping.get(base_id) or base_id

# ---------------------------------------
# 📄 ESG Report Parsing
# ---------------------------------------

def extract_text_docling(pdf_path, min_chunk_chars=300):
    from docling.document_converter import DocumentConverter
    from docling.datamodel.pipeline_options import PdfPipelineOptions
    from docling.document_converter import PdfFormatOption, InputFormat
    from docling.backend.docling_parse_v2_backend import DoclingParseV2DocumentBackend

    pipeline_options = PdfPipelineOptions(do_ocr=False, do_table_structure=False)

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
                backend=DoclingParseV2DocumentBackend
            )
        }
    )

    result = doc_converter.convert(pdf_path)
    full_text = result.document.export_to_markdown()

    buffer, chunks = "", []
    for line in full_text.splitlines():
        buffer += line.strip() + " "
        if len(buffer) >= min_chunk_chars:
            chunks.append(buffer.strip())
            buffer = ""
    if buffer:
        chunks.append(buffer.strip())
    return full_text, chunks


# ---------------------------------------
# 🧠 Vectorstore Builders
# ---------------------------------------

def build_disclosure_rule_vectorstore(disclosures, path="disclosure_rule_vectorstore", force=False):
    from langchain.vectorstores import FAISS
    from langchain.schema import Document
    from langchain_openai import OpenAIEmbeddings

    embedding = OpenAIEmbeddings(model=EMBED_MODEL)
    if not os.path.exists(path) or force:
        docs = [
            Document(page_content=f"{d['title']}\n\n{d['description']}", metadata={"id": d["id"]})
            for d in disclosures if d.get("description")
        ]
        store = FAISS.from_documents(docs, embedding)
        store.save_local(path)
    else:
        store = FAISS.load_local(path, embedding, allow_dangerous_deserialization=True)
    return store

def build_runtime_state_index(results: dict, rebuild=False):
    from langchain.vectorstores import FAISS
    from langchain.schema import Document
    from langchain_openai import OpenAIEmbeddings
    import json, os

    FAISS_INDEX_PATH = "faiss_global_index"
    embedding_model = OpenAIEmbeddings(model=EMBED_MODEL)

    if os.path.exists(FAISS_INDEX_PATH) and not rebuild:
        print("✅ Loading saved FAISS index...")
        return FAISS.load_local(FAISS_INDEX_PATH, embedding_model, allow_dangerous_deserialization=True)

    docs = []
    section_counts = {}

    for section, content in results.items():
        section_doc_count = 0

        if isinstance(content, list):  # e.g., matching
            for item in content:
                if not isinstance(item, dict):
                    continue
                metadata = {
                    "section": section,
                    "id": item.get("id") or item.get("disclosure_id") or item.get("original_id"),
                    "title": item.get("title"),
                    "status": item.get("status"),
                    "rating": item.get("rating"),
                    "reason": item.get("reason"),
                    "source": item.get("source"),
                }
                page_content = json.dumps(item, indent=2, ensure_ascii=False)
                docs.append(Document(page_content=page_content, metadata=metadata))
                section_doc_count += 1

        elif isinstance(content, dict):  # e.g., metadata, evaluation, fixes (keyed by model)
            for model, items in content.items():
                if isinstance(items, dict):
                    items = list(items.values())
                elif not isinstance(items, list):
                    continue

                for item in items:
                    if not isinstance(item, dict):
                        continue
                    metadata = {
                        "section": section,
                        "model": model,
                        "id": item.get("id") or item.get("disclosure_id"),
                        "rating": item.get("rating"),
                        "reason": item.get("reason"),
                    }
                    page_content = json.dumps(item, indent=2, ensure_ascii=False)
                    docs.append(Document(page_content=page_content, metadata=metadata))
                    section_doc_count += 1

        section_counts[section] = section_doc_count


def build_esg_report_index(text_chunks, rebuild=False):
    from langchain.vectorstores import FAISS
    from langchain.schema import Document
    from langchain_openai import OpenAIEmbeddings
    import os

    FAISS_INDEX_PATH = "faiss_content_index"
    embedding = OpenAIEmbeddings(model=EMBED_MODEL)

    if os.path.exists(FAISS_INDEX_PATH) and not rebuild:
        print("✅ Loading saved ESG content index...")
        return FAISS.load_local(FAISS_INDEX_PATH, embedding, allow_dangerous_deserialization=True)

    print("📄 Building ESG content index from chunks...")

    # Create Document objects with chunk_index metadata
    docs = [
        Document(page_content=chunk, metadata={"chunk_index": i})
        for i, chunk in enumerate(text_chunks)
    ]

    index = FAISS.from_documents(docs, embedding)
    index.save_local(FAISS_INDEX_PATH)
    return index


def determine_metadata_source():
    from pathlib import Path

    llm_all_exist = all(Path(f"llm_matching_outputs/llm_matched_{model}.json").exists() for model in SELECTED_MODELS)
    traditional_exists = Path("matched_traditional_best.json").exists()

    if llm_all_exist and traditional_exists:
        print("🧠 Both LLM and Traditional match files are available.")
        print("⚠️ Cannot prompt for input interactively in this environment.")
        print("👉 Defaulting to LLM matches. Set explicitly if needed.")
        return "llm"
    elif llm_all_exist:
        return "llm"
    elif traditional_exists:
        return "traditional"
    return None



# ---------------------------------------
# 🔍 Disclosure Matching
# ---------------------------------------

def safe_description(d):
    return d["description"].strip() if d.get("description", "").strip() else f"This disclosure requires information about: {d['title']}"

#This function gets called by matche_dicslosures function if method ="llm"
    return d["description"].strip() if d.get("description", "").strip() else f"This disclosure requires information about: {d['title']}"
def match_disclosures_with_llm(disclosures, esg_vectorstore, model_label, provider_config, top_k=10, return_raw=False):
    import json
    from tqdm import tqdm
    import os
    from openai import OpenAI
    import anthropic
    import google.generativeai as genai
    from langchain_core.vectorstores import VectorStoreRetriever
    from time import time

    model_info = provider_config[model_label]
    model = model_info["model"]
    provider = model_info["provider"]

    if provider == "openai":
        client = OpenAI()
    elif provider == "claude":
        client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    elif provider == "gemini":
        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        client = genai.GenerativeModel(model)
    else:
        raise ValueError(f"Unsupported provider: {provider}")

    retriever: VectorStoreRetriever = esg_vectorstore.as_retriever(search_kwargs={"k": top_k})
    results = []
    raw_logs = []

    for d in tqdm(disclosures, desc=f"🔍 Matching with {model_label}"):
        query = f"{d.get('title', '')}\n\n{safe_description(d)}"
        docs_with_scores = esg_vectorstore.similarity_search_with_score(query, k=top_k)

        prompt_chunks = []
        chunk_lookup = {}
        for i, (doc, score) in enumerate(docs_with_scores):
            chunk_id = f"Chunk {i+1}"
            chunk_lookup[chunk_id] = doc.page_content
            prompt_chunks.append(f"[{chunk_id}]:\n{doc.page_content.strip()}")

        prompt = f"""
You are an expert sustainability auditor. Your task is to determine which ESG report chunks are relevant to the disclosure requirement below.

Use your understanding of ESG standards to identify which chunks support or demonstrate disclosure of the required information. Prioritize factual alignment, not keyword overlap.

=== DISCLOSURE REQUIREMENT ===
ID: {d.get("id")}
Title: {d.get("title")}
Details:
{d.get("description")}

=== ESG REPORT EXCERPTS ===
Each chunk is labeled. Review them carefully:
{chr(10).join(prompt_chunks)}

=== OUTPUT FORMAT ===
Respond with a valid JSON object in this format:
{{
  "matched_chunks": ["Chunk 2", "Chunk 4"],
  "justification": "Chunk 2 includes packaging KPIs; Chunk 4 discusses recycling systems relevant to the disclosure."
}}

INSTRUCTIONS:
- Match only chunks that directly help fulfill or evidence the disclosure.
- Do not select vague, partial, or unrelated chunks.
- If no chunk is relevant, return an empty list.
- Respond with JSON only. Do not add explanation, markdown, or extra commentary.
""".strip()

        raw_output = ""
        latency = None
        error = None
        usage = {}

        try:
            start_time = time()

            if provider == "openai":
                response = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=1000
                )
                raw_output = response.choices[0].message.content.strip()

            elif provider == "claude":
                response = client.messages.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=1000
                )
                raw_output = response.content[0].text.strip()

            elif provider == "gemini":
                response = client.generate_content(prompt)
                raw_output = response.text.strip()

            latency = round(time() - start_time, 2)
            usage = get_token_usage(response, prompt, raw_output, provider)

            raw_output = raw_output.strip("` \n")
            if raw_output.lower().startswith("json"):
                raw_output = raw_output[4:].strip()

            parsed = json.loads(raw_output)
            matched_chunks = [chunk_lookup[cid] for cid in parsed.get("matched_chunks", []) if cid in chunk_lookup]

            results.append({
                "id": d.get("id"),
                "title": d.get("title", ""),
                "source": d.get("source", "GRI_Topic"),
                "matched_chunks": matched_chunks,
                "status": "matched" if matched_chunks else "no_match"
            })

        except Exception as e:
            error = str(e)
            results.append({
                "id": d.get("id"),
                "title": d.get("title", ""),
                "source": d.get("source", "GRI_Topic"),
                "matched_chunks": [],
                "status": "no_match",
                "error": error
            })

        raw_logs.append({
            "disclosure_id": d.get("id"),
            "title": d.get("title"),
            "provider": provider,
            "model": model,
            "prompt": prompt,
            "response": raw_output,
            "latency_sec": latency,
            **usage,
            "error": error
        })

    return (results, raw_logs) if return_raw else results


def match_disclosures(disclosures, esg_vectorstore, top_k=10, similarity_threshold=1.2, method="llm", provider_config=None, verbose=False):
    from tqdm import tqdm
    from langchain_core.vectorstores import VectorStoreRetriever
    import json
    import os
    from copy import deepcopy

    global GRI_ID_MAP
    os.makedirs("llm_matching_outputs", exist_ok=True)

    if method == "traditional":
        results = []
        retriever: VectorStoreRetriever = esg_vectorstore.as_retriever(search_kwargs={"k": top_k})
        for d in tqdm(disclosures, desc="Matching (Traditional)"):
            raw_id = d.get("id", "UNKNOWN")
            id_ = normalize_gri_id(raw_id, GRI_ID_MAP)
            query = f"{d.get('title', '')}\n\n{safe_description(d)}"
            docs_with_scores = esg_vectorstore.similarity_search_with_score(query, k=top_k)

            matched_chunks = []
            matched_scores = []

            for doc, score in docs_with_scores:
                if score <= similarity_threshold:
                    matched_chunks.append(doc.page_content)
                    matched_scores.append(float(score))

            results.append({
                "original_id": raw_id,
                "id": id_,
                "title": d.get("title", ""),
                "description": d.get("description", ""),
                "source": d.get("source", ""),
                "matched_chunks": matched_chunks,
                "similarity_scores": matched_scores,
                "status": "matched" if matched_chunks else "no_match"
            })

        with open("matched_traditional_best.json", "w") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        with open("matched_only.json", "w") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        return results

    elif method == "llm":
        assert provider_config is not None, "❌ LLM mode requires provider_config"

        all_results = {}
        all_raw_logs = {}

        for model_name, config in provider_config.items():
            print(f"\n🤖 Matching with {model_name.upper()}")

            # Capture both results and raw logs
            results, raw_logs = match_disclosures_with_llm(
                disclosures=deepcopy(disclosures),
                esg_vectorstore=esg_vectorstore,
                model_label=model_name,
                provider_config={model_name: config},
                top_k=top_k,
                return_raw=True  # Ensure your inner function supports this
            )
            all_results[model_name] = results
            all_raw_logs[model_name] = raw_logs

            with open(f"llm_matching_outputs/llm_matched_{model_name}.json", "w") as f:
                json.dump(results, f, indent=2, ensure_ascii=False)

            with open(f"llm_matching_outputs/llm_raw_matching_{model_name}.json", "w") as f:
                json.dump(raw_logs, f, indent=2, ensure_ascii=False)

        return all_results

    else:
        raise ValueError("❌ Unknown matching method. Use 'traditional' or 'llm'.")



# ---------------------------------------
# 💾 Save Helpers
# ---------------------------------------

def convert_to_serializable(obj):
    import numpy as np

    if isinstance(obj, (float, int, str, bool, type(None))):
        return obj
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    if hasattr(obj, "tolist"):  # e.g., np.ndarray
        return obj.tolist()
    return str(obj)



def save_matches(matches, filename="matched_disclosures_full.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(convert_to_serializable(matches), f, indent=2, ensure_ascii=False)

def save_matched_only(results, output_path="matched_only.json"):
    matched = [r for r in results if r.get("status") == "matched"]
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(convert_to_serializable(matched), f, indent=2, ensure_ascii=False)

# ---------------------------------------
# 🤖 Agentic AI Runtime: Metadata & Fixes
# ---------------------------------------
def run_metadata_extraction(all_disclosures, providers, selected_models, extraction_match_source: str = "llm"):
    import anthropic
    import google.generativeai as genai
    from openai import OpenAI
    import os, json, time
    from tqdm import tqdm

    os.makedirs("metadata_outputs", exist_ok=True)

    for label in selected_models:
        opts = providers[label]
        results = {}
        raw_log = {}

        # Initialize model client
        if opts["provider"] == "openai":
            client = OpenAI()
        elif opts["provider"] == "claude":
            client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        elif opts["provider"] == "gemini":
            genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
            model = genai.GenerativeModel(opts["model"])
        else:
            raise ValueError("Unsupported provider")

        for d in tqdm(all_disclosures, desc=f"🔎 {label} Metadata Extraction Processing"):
            if d.get("status") != "matched" or not d.get("matched_chunks"):
                continue

            context = "\n\n---\n\n".join(d["matched_chunks"])
            prompt = build_metadata_prompt(d, context)
            disclosure_id = d["id"]

            start_time = time.time()
            output = ""
            res = None

            try:
                if opts["provider"] == "openai":
                    res = client.chat.completions.create(
                        model=opts["model"],
                        messages=[{"role": "user", "content": prompt}],
                        temperature=0.2,
                        max_tokens=2000
                    )
                    output = res.choices[0].message.content.strip()

                elif opts["provider"] == "claude":
                    res = client.messages.create(
                        model=opts["model"],
                        messages=[{"role": "user", "content": prompt}],
                        temperature=0.2,
                        max_tokens=1024
                    )
                    output = res.content[0].text.strip()

                elif opts["provider"] == "gemini":
                    res = model.generate_content(prompt)
                    output = res.text.strip("`\n ")

                latency = round(time.time() - start_time, 2)
                usage = get_token_usage(res, prompt, output, opts["provider"])

                parsed = extract_json_from_text(output, provider=opts["provider"])
                parsed["disclosure_id"] = disclosure_id
                results[disclosure_id] = parsed

                raw_log[disclosure_id] = {
                    "prompt": prompt,
                    "output": output,
                    **usage,
                    "latency_sec": latency
                }

            except Exception as e:
                results[disclosure_id] = {"error": str(e)}
                raw_log[disclosure_id] = {
                    "prompt": prompt,
                    "output": f"[LLM Error: {e}]"
                }

        method_tag = extraction_match_source.lower()
        with open(f"metadata_outputs/esg_metadata_{label.lower()}_from_{method_tag}.json", "w") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        with open(f"metadata_outputs/esg_metadata_{label.lower()}_raw_responses_{method_tag}.json", "w") as f:
            json.dump(raw_log, f, indent=2, ensure_ascii=False)


def run_llm_metadata_evaluation(disclosures, metadata_by_model, model_label, model_config, output_prefix=None):
    import anthropic
    import google.generativeai as genai
    from openai import OpenAI
    import os
    import json
    import time

    results = {}
    raw_log = {}

    if model_config["provider"] == "openai":
        client = OpenAI()
    elif model_config["provider"] == "claude":
        client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    elif model_config["provider"] == "gemini":
        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        client = genai.GenerativeModel(model_config["model"])
    else:
        raise ValueError("Unsupported provider")

    for d in tqdm(disclosures, desc=f"Evaluating {model_label}"):
        disclosure_id = d["id"]
        metadata = metadata_by_model.get(disclosure_id)

        if not metadata or "error" in metadata:
            results[disclosure_id] = {
                "disclosure_id": disclosure_id,
                "rating": "Non-Compliant",
                "score": 0.0,
                "justification": "Metadata not available or extraction failed."
            }
            raw_log[disclosure_id] = "[Missing metadata]"
            continue

        prompt = build_metadata_eval_prompt(d, metadata)
        start_time = time.time()
        output = ""
        res = None

        try:
            if model_config["provider"] == "openai":
                res = client.chat.completions.create(
                    model=model_config["model"],
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=500
                )
                output = res.choices[0].message.content.strip()

            elif model_config["provider"] == "claude":
                res = client.messages.create(
                    model=model_config["model"],
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=500
                )
                output = res.content[0].text.strip()

            elif model_config["provider"] == "gemini":
                res = client.generate_content(prompt)
                output = res.text.strip("`\n ")
                if output.lower().startswith("json"):
                    output = output[output.find("{"):].strip()

            else:
                raise ValueError("Unsupported provider")

            latency = round(time.time() - start_time, 2)
            usage = get_token_usage(res, prompt, output, model_config["provider"])

            parsed = json.loads(output)
            parsed["disclosure_id"] = disclosure_id

            if "rating" not in parsed or "score" not in parsed or "justification" not in parsed:
                raise ValueError("Incomplete fields in LLM response")

            results[disclosure_id] = parsed
            raw_log[disclosure_id] = {
                "prompt": prompt,
                "output": output,
                "latency_sec": latency,
                **usage
            }

        except Exception as e:
            latency = round(time.time() - start_time, 2)
            results[disclosure_id] = {
                "disclosure_id": disclosure_id,
                "rating": "Non-Compliant",
                "score": 0.0,
                "justification": f"[LLM Error: {e}]"
            }
            raw_log[disclosure_id] = {
                "prompt": prompt,
                "output": output if 'output' in locals() else "[No output]",
                "latency_sec": latency,
                "error": str(e)
            }

    os.makedirs("metadata_outputs", exist_ok=True)
    output_prefix = output_prefix or f"metadata_outputs/metadata_eval_{model_label}"

    with open(f"{output_prefix}.json", "w") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    with open(f"{output_prefix}_raw.json", "w") as f:
        json.dump(raw_log, f, indent=2, ensure_ascii=False)

#Note: Consider removing the vector store and retrieving the disclosure directly
def generate_esg_fixes_with_retrieval_multi(compliance_report, rule_vectorstore, model_configs, selected_models, return_raw=False, fixes_match_source=None):
    import anthropic
    import google.generativeai as genai
    from openai import OpenAI
    import time
    import os, json
    from tqdm import tqdm

    os.makedirs("fix_outputs", exist_ok=True)
    all_raw_logs = []

    for model in selected_models:
        config = model_configs[model]

        if config["provider"] == "openai":
            client = OpenAI()
        elif config["provider"] == "claude":
            client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        elif config["provider"] == "gemini":
            genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
            client = genai.GenerativeModel(config["model"])
        else:
            raise ValueError("Unsupported provider")

        results = []
        raw_logs = []

        for item in tqdm(compliance_report, desc=f"{model.upper()} Fixes"):
            if item.get("rating") not in {"Non-Compliant", "Partially Compliant"}:
                continue

            disclosure_id = item.get("disclosure_id")
            reason = item.get("reason", "No reason provided")
            disclosure_title = item.get("title", f"Disclosure {disclosure_id}")

            retrieved = rule_vectorstore.similarity_search(disclosure_id, k=1)
            rule_text = retrieved[0].page_content if retrieved else "[No rule found]"
            prompt = build_fix_suggestion_prompt(disclosure_id, item["rating"], reason, rule_text)

            start_time = time.time()
            suggestion = "[No output]"
            res = None

            try:
                if config["provider"] == "openai":
                    res = client.chat.completions.create(
                        model=config["model"],
                        messages=[{"role": "user", "content": prompt}],
                        temperature=0.2,
                        max_tokens=300
                    )
                    suggestion = res.choices[0].message.content.strip()

                elif config["provider"] == "claude":
                    res = client.messages.create(
                        model=config["model"],
                        messages=[{"role": "user", "content": prompt}],
                        temperature=0.2,
                        max_tokens=300
                    )
                    suggestion = res.content[0].text.strip()

                elif config["provider"] == "gemini":
                    res = client.generate_content(prompt)
                    suggestion = res.text.strip("`\n ")

                latency = round(time.time() - start_time, 2)
                usage = get_token_usage(res, prompt, suggestion, config["provider"])

            except Exception as e:
                suggestion = f"[ERROR: {e}]"
                latency = round(time.time() - start_time, 2)
                usage = {"input_tokens": None, "output_tokens": None}

            results.append({
                "disclosure_id": disclosure_id,
                "title": disclosure_title,
                "reason": reason,
                "rating": item["rating"],
                "suggested_fix": suggestion
            })

            raw_logs.append((disclosure_id, {
                "prompt": prompt,
                "output": suggestion,
                "latency_sec": latency,
                **usage,
                "rating": item["rating"]
            }))

        suffix = f"_from_{fixes_match_source}" if fixes_match_source else ""

        with open(f"fix_outputs/esg_fixes_{model}{suffix}.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        raw_log_dict = {k: v for k, v in raw_logs}
        with open(f"fix_outputs/esg_fixes_{model}{suffix}_raw.json", "w", encoding="utf-8") as f:
            json.dump(raw_log_dict, f, indent=2)

        all_raw_logs.extend(raw_logs)

    if return_raw:
        return all_raw_logs


In [ ]:
import os

def print_tree(start_path='.', indent=''):
    for item in os.listdir(start_path):
        path = os.path.join(start_path, item)
        print(indent + '|-- ' + item)
        if os.path.isdir(path):
            print_tree(path, indent + '    ')

print_tree()


# Preparing Admin Side Uploads

In [ ]:
# Upload the mapping file
uploaded = files.upload()
mapping_path = list(uploaded.keys())[0]

# Run this admin utility
admin_load_gri_mapping_and_save(mapping_path)


In [ ]:
def upload_and_extract_disclosure_zips():
    """
    Allow multiple ZIP uploads for GRI and SASB JSONs.
    Returns two lists: all_gri_files, all_sasb_files
    """

    all_gri_files = []
    all_sasb_files = []

    print("📁 Upload GRI and SASB ZIPs one at a time. Type 'no' to stop.\n")

    while True:
        print("🔼 Upload a ZIP file:")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ No file uploaded.")
            break

        zip_name = list(uploaded.keys())[0]
        extract_dir = f"/content/tmp_extracted_{zip_name.replace('.zip','')}"
        os.makedirs(extract_dir, exist_ok=True)

        with zipfile.ZipFile(zip_name, "r") as zip_ref:
            zip_ref.extractall(extract_dir)

        # Classify files based on filename pattern
        for root, _, files_in_dir in os.walk(extract_dir):
            for f in files_in_dir:
                if not f.endswith(".json"):
                    continue
                full_path = os.path.join(root, f)

                if f.startswith("SASB_") or "sasb" in f.lower():
                    all_sasb_files.append(full_path)
                elif f.startswith("GRI_") or "gri" in f.lower():
                    all_gri_files.append(full_path)

        print(f"✅ Extracted from {zip_name}: GRI: {len(all_gri_files)}, SASB: {len(all_sasb_files)}")

        cont = input("➕ Upload another ZIP? (yes/no): ").strip().lower()
        if cont in ["no", "n"]:
            break

    print(f"\n📂 Total GRI files: {len(all_gri_files)} | SASB files: {len(all_sasb_files)}")
    return all_gri_files, all_sasb_files


def load_disclosures(gri_json_files, sasb_json_files):
    DISCLOSURES = []

    # === GRI files ===
    for path in gri_json_files:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Failed to load GRI JSON: {path} — {e}")
            continue

        fname = os.path.basename(path)

        # GRI 3 (structure → disclosures)
        if "GRI_3" in fname or "GRI 3" in fname:
            if "structure" in data and "disclosures" in data["structure"]:
                for entry in data["structure"]["disclosures"]:
                    DISCLOSURES.append({
                        "id": f"3-{entry['id'].split('-')[-1]}",
                        "title": entry["title"],
                        "description": "\n".join(entry.get("requirements", [])),
                        "source": "GRI_3"
                    })

        # GRI 2 (standards list)
        if "GRI_2" in fname or "GRI 2" in fname:
            for item in data.get("standards", []):
                DISCLOSURES.append({
                    "id": item["code"],
                    "title": item["title"],
                    "description": item.get("description", ""),
                    "source": "GRI_2"
                })

        # GRI Topic Standards (main enhancement here)
        if "GRI" in fname:
            disclosures_found = False
            standard = data.get("standard", {})
            sources = [
                data.get("topic_disclosures", []),
                data.get("disclosures", [])
            ]

            if isinstance(standard, dict):
                sources += [
                    standard.get("topic_disclosures", []),
                    standard.get("disclosures", []),
                    standard.get("management_disclosure", {}).get("disclosures", [])
                ]

            for source in sources:
                for item in source:
                    raw_id = item.get("id", "UNKNOWN")

                    # 🔧 Enhanced description construction
                    description_parts = []

                    if "description" in item and item["description"].strip():
                        description_parts.append(item["description"].strip())

                    if "narratives" in item and isinstance(item["narratives"], list):
                        description_parts.append(" ".join(item["narratives"]))

                    if "requirements" in item and isinstance(item["requirements"], list):
                        description_parts.append("Requirements: " + " ".join(item["requirements"]))

                    if "guidance" in item and isinstance(item["guidance"], list):
                        description_parts.append("Guidance: " + " ".join(item["guidance"]))

                    combined_description = "\n\n".join(description_parts)

                    DISCLOSURES.append({
                        "original_id": raw_id,
                        "id": raw_id,
                        "title": item.get("title", ""),
                        "description": combined_description,
                        "source": "GRI_Topic"
                    })

                    disclosures_found = True

            if not disclosures_found:
                print(f"⚠️ Could not extract disclosures from {fname}")

    # === SASB files ===
    for path in sasb_json_files:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Failed to load SASB JSON: {path} — {e}")
            continue

        sector = data.get("sector", "Unknown Sector")

        for topic in data.get("disclosure_topics", []):
            topic_name = topic.get("topic", "Untitled Topic")
            for metric in topic.get("metrics", []):
                raw_id = metric.get("id", "UNKNOWN")
                DISCLOSURES.append({
                    "original_id": raw_id,
                    "id": raw_id,
                    "title": f"{topic_name} – {raw_id}",
                    "description": metric.get("description", ""),
                    "source": f"SASB ({sector})"
                })

        for metric in data.get("activity_metrics", []):
            raw_id = metric.get("id", "UNKNOWN")
            DISCLOSURES.append({
                "original_id": raw_id,
                "id": raw_id,
                "title": f"Activity Metric – {raw_id}",
                "description": metric.get("description", ""),
                "source": f"SASB ({sector})"
            })

    print(f"✅ Loaded {len(DISCLOSURES)} total disclosures.")
    return DISCLOSURES


def admin_prepare_and_save_disclosures_full():
    """
    Admin step: Upload and load *all* GRI and SASB disclosures (no filtering).
    Saves to admin/all_disclosures.json for user runtime.
    """
    print("📦 Upload GRI and SASB ZIP files now...")
    gri_files, sasb_files = upload_and_extract_disclosure_zips()

    # ✅ Load everything, no sector filter
    DISCLOSURES = load_disclosures(gri_files, sasb_files)

    os.makedirs("admin", exist_ok=True)
    with open("admin/all_disclosures.json", "w", encoding="utf-8") as f:
        json.dump(DISCLOSURES, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved {len(DISCLOSURES)} total disclosures from GRI and SASB to admin/all_disclosures.json")


In [ ]:
admin_prepare_and_save_disclosures_full()

In [ ]:
!ls -lh admin/

# Agentic AI Tools Definition

In [ ]:
from langchain.agents import tool
from pydantic import BaseModel, Field
from datetime import datetime
import os, json, time
from pathlib import Path
from google.colab import files
import functools
from typing import Optional


# === Runtime logging ===
TOOL_LATENCY_LOG = {}
LLM_COST_LOG = {}

# === Cost rates (per million tokens) ===
LLM_PRICING = {
    "gpt": {"input_price": 0.10 / 1e6, "output_price": 0.40 / 1e6},
    "claude": {"input_price": 0.80 / 1e6, "output_price": 4.00 / 1e6},
    "gemini": {"input_price": 0.10 / 1e6, "output_price": 0.40 / 1e6},
}


def track_latency(tool_name):
    def wrapper(func):
        @functools.wraps(func)  # ✅ preserves docstring, name, etc.
        def timed(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            TOOL_LATENCY_LOG[tool_name] = round(time.time() - start, 2)
            return result
        return timed
    return wrapper


@tool(return_direct=True)
@track_latency("tool_upload_esg_pdf")
def tool_upload_esg_pdf() -> str:
    """Upload and extract an ESG report PDF using Docling pipeline. Automatically resets pipeline state."""
    import os
    import shutil
    from pathlib import Path
    from google.colab import files  # if in Colab, else adjust

    # 🔄 Hard reset of pipeline globals
    global full_text, ESG_CHUNKS, ESG_VECTORSTORE
    global DISCLOSURES, MATCH_RESULTS, GLOBAL_STATE_INDEX
    global RESULTS
    global selected_gri, selected_sasb

    full_text = None
    ESG_CHUNKS = None
    ESG_VECTORSTORE = None
    DISCLOSURES = []
    MATCH_RESULTS = []
    GLOBAL_STATE_INDEX = None
    RESULTS = {
        "matching": [],
        "metadata": {},
        "evaluation": {},
        "fixes": {}
    }
    selected_gri = False
    selected_sasb = False

    # 🧹 Clear all output directories (full folder deletion)
    for folder in ["faiss_content_index", "faiss_global_index", "llm_matching_outputs", "metadata_outputs", "fix_outputs"]:
        try:
            if os.path.isdir(folder):
                shutil.rmtree(folder)
                print(f"🧹 Cleared directory: {folder}")
            elif os.path.exists(folder):
                os.remove(folder)
                print(f"🧹 Removed file: {folder}")
        except Exception as e:
            print(f"⚠️ Could not clear {folder}: {e}")

    # 📤 Prompt user to upload
    print("📤 Please upload your ESG PDF report...")
    uploaded = files.upload()
    if not uploaded:
        return "❌ No file uploaded."

    path = list(uploaded.keys())[0]
    print(f"✅ File uploaded: {path}")

    # 🔍 Run extraction
    try:
        full_text, ESG_CHUNKS = extract_text_docling(path)
        if not ESG_CHUNKS or not full_text:
            return "⚠️ Extraction failed: No content extracted from the PDF."

        ESG_VECTORSTORE = build_esg_report_index(ESG_CHUNKS, rebuild=True)
        print(f"🧠 ESG content index built with {len(ESG_CHUNKS)} chunks.")

        return f"""✅ ESG report extracted and chunked. {len(ESG_CHUNKS)} chunks generated from: {path}"""

    except Exception as e:
        return f"❌ Failed to extract PDF: {e}"



# === 2. Select Guideline + Sector ===

from langchain.agents import tool
from pydantic import BaseModel, Field
import os, json, time

@tool(return_direct=True)
def tool_list_framework_options(_: str = "") -> str:
    """List the available ESG reporting frameworks supported by this pipeline."""
    return (
        "📚 Available ESG Framework Options:\n"
        "1. GRI (Global Reporting Initiative)\n"
        "2. SASB (Sustainability Accounting Standards Board)\n"
        "3. Both GRI and SASB\n\n"
    )

class FrameworkInput(BaseModel):
    choice: str = Field(..., description="1 = GRI only, 2 = SASB only, 3 = Both GRI and SASB")

@tool(args_schema=FrameworkInput, return_direct=True)
@track_latency("tool_select_framework")
def tool_select_framework(choice: str) -> str:
    """Step 1: User selects ESG framework(s)."""
    global selected_gri, selected_sasb, DISCLOSURES

    if not os.path.exists("admin/all_disclosures.json"):
        return "❌ Disclosures not prepared by admin."

    all_disclosures = load_admin_disclosures()

    selected_gri = selected_sasb = False
    DISCLOSURES = []

    response = []

    if choice == "1":
        selected_gri = True
    elif choice == "2":
        selected_sasb = True
    elif choice == "3":
        selected_gri = selected_sasb = True
    else:
        return "❌ Invalid input. Please choose 1 (GRI), 2 (SASB), or 3 (Both)."

    if selected_gri:
        gri_disclosures = [d for d in all_disclosures if "gri" in d["source"].lower()]
        DISCLOSURES.extend(gri_disclosures)
        response.append(f"✅ Loaded {len(gri_disclosures)} GRI disclosures.")

    if selected_sasb:
        response.append("📊 SASB selected. Please choose your sector(s) to continue.")
        # 🧠 Inline sector listing
        sectors = sorted({
            d["source"].split("(", 1)[-1].rstrip(")")
            for d in all_disclosures if "sasb" in d["source"].lower()
        })
        if sectors:
            response.append("📋 Available SASB sectors:\n" + "\n".join([f"[{i}] {s}" for i, s in enumerate(sectors)]))
        else:
            response.append("⚠️ No SASB sectors found.")

    return "\n\n".join(response)





@tool(return_direct=True)
@track_latency("tool_list_sasb_sectors")
def tool_list_sasb_sectors() -> str:
    """Lists available SASB sectors (for use with tool_select_sasb_sectors)."""
    if not os.path.exists("admin/all_disclosures.json"):
        return "❌ Disclosures not found."

    all_disclosures = load_admin_disclosures()
    sectors = sorted({
        d["source"].split("(", 1)[-1].rstrip(")")
        for d in all_disclosures if "sasb" in d["source"].lower()
    })

    if not sectors:
        return "⚠️ No SASB sectors found."

    return "📊 Available SASB sectors:\n" + "\n".join([f"[{i}] {name}" for i, name in enumerate(sectors)])
from langchain.agents import tool
from pydantic import BaseModel, Field
from typing import List

class SasbSectorInput(BaseModel):
    sectors: List[int] = Field(..., description="List of SASB sector indices. Example: [0, 2, 4]")

@tool(args_schema=SasbSectorInput, return_direct=True)
@track_latency("tool_select_sasb_sectors")
def tool_select_sasb_sectors(sectors: List[int]) -> str:
    """Select one or more SASB sectors after enabling SASB framework."""
    global selected_sasb, DISCLOSURES

    if not selected_sasb:
        return "❌ SASB was not selected. Please select the framework first."

    if not os.path.exists("admin/all_disclosures.json"):
        return "❌ Disclosures not found."

    all_disclosures = load_admin_disclosures()
    all_sectors = sorted({
        d["source"].split("(", 1)[-1].rstrip(")")
        for d in all_disclosures if "sasb" in d["source"].lower()
    })

    try:
        selected_names = [all_sectors[i] for i in sectors if 0 <= i < len(all_sectors)]
    except Exception:
        return "❌ Invalid sector indices provided."

    DISCLOSURES.extend([
        d for d in all_disclosures
        if any(sector.lower() in d["source"].lower() for sector in selected_names)
    ])

    return f"""✅ SASB sectors selected: {", ".join(selected_names)}.
📄 Total disclosures loaded: {len(DISCLOSURES)}.

Would you like to begin disclosure-to-report semantic matching?
You may choose:
- 🤖 LLM-based (contextual understanding)
- 🧮 Traditional (pattern-based similarity)
"""
from langchain_openai import OpenAIEmbeddings  # ✅ Updated import

class MatchInput(BaseModel):
    method: Optional[str] = Field(None, description="Specify 'llm' or 'traditional' for matching method.")

@tool(args_schema=MatchInput, return_direct=True)
@track_latency("tool_run_matching")
def tool_run_matching(method: Optional[str] = None) -> str:
    """
    Run disclosure-to-report semantic matching using either 'llm' or 'traditional' methods.

    - 'llm': Uses GPT-based contextual understanding (default if selected explicitly).
    - 'traditional': Embedding-based cosine similarity (faster, no model usage).

    If no method is provided, the agent will request a selection.
    """
    from pathlib import Path
    import json
    import os

    global ESG_CHUNKS, DISCLOSURES, ESG_VECTORSTORE, MATCH_RESULTS

    if not ESG_CHUNKS:
        return "❌ ESG chunks not found. Please upload and extract the ESG report first."
    if not DISCLOSURES:
        return "❌ Disclosures not loaded. Please select GRI/SASB guideline first."

    if method is None:
        return (
            "❓ Which matching method would you like to use?\n\n"
            "**Options:**\n"
            "- `llm`: 🤖 LLM-powered semantic matching (context-aware)\n"
            "- `traditional`: 🧮 Embedding-based similarity (faster, no model usage)\n\n"
            "Please type your preferred method to proceed."
        )

    method = method.lower().strip()
    if method not in ["llm", "traditional"]:
        return "❌ Invalid method. Choose either `'llm'` or `'traditional'`."

    try:
        print("🔍 Initializing ESG vectorstore for matching...")
        embedding = OpenAIEmbeddings(model=EMBED_MODEL)
        docs = [Document(page_content=c, metadata={"chunk_index": i}) for i, c in enumerate(ESG_CHUNKS)]
        ESG_VECTORSTORE = FAISS.from_documents(docs, embedding)

        print(f"🚀 Matching method selected: {method.upper()}")
        MATCH_RESULTS = match_disclosures(
            disclosures=DISCLOSURES,
            esg_vectorstore=ESG_VECTORSTORE,
            top_k=10,
            similarity_threshold=1.2,
            method=method,
            provider_config=providers if method == "llm" else None,
            verbose=False
        )

        save_matches(MATCH_RESULTS)
        if method == "traditional":
            save_matched_only(MATCH_RESULTS)

        RESULTS["matching"] = MATCH_RESULTS
        global GLOBAL_STATE_INDEX
        GLOBAL_STATE_INDEX = build_runtime_state_index(RESULTS, rebuild=True)

        # === Matching Summary ===
        if method == "llm":
            summary_lines = []
            total_matched = 0
            total_count = 0

            for model, matches in MATCH_RESULTS.items():
                matched = sum(1 for r in matches if r.get("status") == "matched")
                count = len(matches)
                total_matched += matched
                total_count += count
                summary_lines.append(f"🤖 Model **{model.upper()}**: {matched} of {count} matched.")

            summary = "\n".join(summary_lines)
            preview = MATCH_RESULTS[next(iter(MATCH_RESULTS))][0] if MATCH_RESULTS else {}

        else:
            matched = sum(1 for r in MATCH_RESULTS if r.get("status") == "matched")
            total = len(MATCH_RESULTS)
            summary = f"📊 Traditional matching: {matched} of {total} matched."
            preview = MATCH_RESULTS[0] if MATCH_RESULTS else {}

        preview_clean = {
            k: v for k, v in preview.items()
            if isinstance(v, (str, int, float, list, dict)) and k != "matched_chunks"
        }
        preview_str = json.dumps(preview_clean, indent=2, ensure_ascii=False) if preview_clean else "(No preview available)"

        method_description = (
            "🔍 **LLM-based matching**: Uses GPT for context-aware disclosure alignment."
            if method == "llm" else
            "📊 **Traditional matching**: Uses embedding similarity (fast, no LLMs)."
        )

        return f"""✅ Matching completed successfully.

{summary}

{method_description}

📁 Output saved to: {'llm_matching_outputs/' if method == 'llm' else ''}matched_only.json

🧾 Preview of first matched disclosure:
{preview_str}
"""

    except Exception as e:
        return f"❌ Matching failed due to: {str(e)}"







# === 4. Metadata Extraction ===
from pydantic import BaseModel, Field
from typing import Optional

class ExtractInput(BaseModel):
    extraction_match_source: Optional[str] = Field(
        None, description="Choose 'llm' for AI-assisted extraction, or 'traditional' for rule-based extraction."
    )

@tool(args_schema=ExtractInput, return_direct=True)
@track_latency("tool_extract_metadata")
def tool_extract_metadata(extraction_match_source: Optional[str] = None) -> str:
    """
    Extract metadata for ESG disclosures using either:
    - 'llm': AI-assisted extraction based on contextual understanding.
    - 'traditional': Pattern- or rule-based methods on pre-matched chunks.
    """
    from pathlib import Path
    import json
    import os

    global SELECTED_MODELS, providers
    if not SELECTED_MODELS:
        return "❌ No models selected. Please run `select_pipeline_models()` first."

    if extraction_match_source is None:
        return (
            "❓ Please specify the match source for metadata extraction:\n\n"
            "- `llm`: Use language models for contextual metadata extraction.\n"
            "- `traditional`: Use rule-based pattern extraction on matched chunks.\n\n"
        )

    extraction_match_source = extraction_match_source.lower().strip()
    if extraction_match_source not in ["llm", "traditional"]:
        return "❌ Invalid choice. Please use 'llm' or 'traditional'."

    print(f"📌 Metadata extraction method: {extraction_match_source.upper()}")

    results = []

    if extraction_match_source == "traditional":
        match_path = "matched_traditional_best.json"
        if not Path(match_path).exists():
            return "❌ Traditional match results not found."

        with open(match_path, "r", encoding="utf-8") as f:
            matched_disclosures = json.load(f)

        if not any(d.get("matched_chunks") for d in matched_disclosures):
            return "⚠️ No matched chunks found in traditional results."

        run_metadata_extraction(matched_disclosures, providers, SELECTED_MODELS, extraction_match_source="traditional")

        for model in SELECTED_MODELS:
            output_path = f"metadata_outputs/esg_metadata_{model}_from_traditional.json"
            if Path(output_path).exists():
                with open(output_path, "r", encoding="utf-8") as f:
                    metadata = json.load(f)
                RESULTS["metadata"][model] = metadata
                sample = next(iter(metadata.values()), {})

                try:
                    preview = json.dumps(sample, indent=2, ensure_ascii=False) if sample else "(no output)"
                except Exception as e:
                    preview = f"[Error rendering preview: {e}]"

                results.append(f"""✅ Metadata extracted with model: {model}
📄 Output file: {output_path}

📌 Method: 🔹 Traditional (pattern-based extraction)
🔍 Preview of first result:
{preview}
""")
            else:
                results.append(f"⚠️ No metadata file found for model: {model}")

    elif extraction_match_source == "llm":
        for model in SELECTED_MODELS:
            path = f"llm_matching_outputs/llm_matched_{model}.json"
            if not Path(path).exists():
                results.append(f"❌ LLM match file not found for model: {model} ({path})")
                continue

            with open(path, "r", encoding="utf-8") as f:
                matched_disclosures = json.load(f)

            if not any(d.get("matched_chunks") for d in matched_disclosures):
                results.append(f"⚠️ No matched chunks found for {model}. Skipping.")
                continue

            run_metadata_extraction(matched_disclosures, providers, [model], extraction_match_source="llm")

            output_path = f"metadata_outputs/esg_metadata_{model}_from_llm.json"
            if Path(output_path).exists():
                with open(output_path, "r", encoding="utf-8") as f:
                    metadata = json.load(f)
                RESULTS["metadata"][model] = metadata
                sample = next(iter(metadata.values()), {})

                try:
                    preview = json.dumps(sample, indent=2, ensure_ascii=False) if sample else "(no output)"
                except Exception as e:
                    preview = f"[Error rendering preview: {e}]"

                results.append(f"""✅ Metadata extracted with model: {model}
📄 Output file: {output_path}

📌 Method: 🤖 LLM-based (AI-assisted extraction)
🔍 Preview of first result:
{preview}
""")
            else:
                results.append(f"⚠️ No metadata file found for model: {model}")

    global GLOBAL_STATE_INDEX
    GLOBAL_STATE_INDEX = build_runtime_state_index(RESULTS, rebuild=True)

    return "\n\n".join(results) if results else "⚠️ No metadata was extracted for any model."



# === 5. Metadata Evaluation ===@tool(return_direct=True)
from pydantic import BaseModel, Field
from typing import Optional

class EvaluateInput(BaseModel):
    evaluation_match_source: Optional[str] = Field(
        None, description="Choose 'llm' for LLM-evaluated metadata or 'traditional' for rule-based metadata evaluation."
    )

@tool(args_schema=EvaluateInput, return_direct=True)
@track_latency("tool_evaluate_metadata")
def tool_evaluate_metadata(evaluation_match_source: Optional[str] = None) -> str:
    """
    Evaluate the quality and completeness of extracted ESG metadata.

    Supported methods:
    - 'llm': Evaluate AI-generated metadata using large language models.
    - 'traditional': Evaluate pattern-based metadata using static heuristics.

    If unspecified, the method will be inferred automatically based on available files.
    """
    from pathlib import Path
    import json
    import os

    global SELECTED_MODELS, RESULTS, GLOBAL_STATE_INDEX
    if not SELECTED_MODELS:
        return "❌ No models selected. Please run `select_pipeline_models()` first."

    if "evaluation" not in RESULTS or not isinstance(RESULTS["evaluation"], dict):
        RESULTS["evaluation"] = {}

    if evaluation_match_source is None:
        return (
            "❓ Please specify the evaluation method:\n\n"
            "- `llm`: Uses AI models to score ESG metadata coverage and compliance.\n"
            "- `traditional`: Applies rule-based logic to structured metadata.\n\n"
        )

    evaluation_match_source = evaluation_match_source.lower().strip()
    if evaluation_match_source not in ["llm", "traditional"]:
        return "❌ Invalid choice. Please use 'llm' or 'traditional'."

    print(f"📌 Evaluation method: {evaluation_match_source.upper()}")

    results = []

    for model in SELECTED_MODELS:
        metadata_path = f"metadata_outputs/esg_metadata_{model}_from_{evaluation_match_source}.json"
        if not Path(metadata_path).exists():
            msg = f"❌ Metadata not found for model: {model} at {metadata_path}"
            print(msg)
            results.append(msg)
            continue

        with open(metadata_path, "r", encoding="utf-8") as f:
            metadata = json.load(f)

        match_path = (
            "matched_only.json" if evaluation_match_source == "traditional"
            else f"llm_matching_outputs/llm_matched_{model}.json"
        )

        if not Path(match_path).exists():
            msg = f"❌ Disclosure match file not found: {match_path}"
            print(msg)
            results.append(msg)
            continue

        with open(match_path, "r", encoding="utf-8") as f:
            matched_disclosures = json.load(f)

        output_base = f"metadata_outputs/metadata_eval_{model}_from_{evaluation_match_source}"
        run_llm_metadata_evaluation(
            disclosures=matched_disclosures,
            metadata_by_model=metadata,
            model_label=model,
            model_config=providers[model],
            output_prefix=output_base
        )

        result_path = f"{output_base}.json"
        if Path(result_path).exists():
            with open(result_path, "r", encoding="utf-8") as f:
                result_data = json.load(f)

            RESULTS["evaluation"][model] = result_data
            sample = next(iter(result_data.values()), {})

            try:
                preview = json.dumps(sample, indent=2, ensure_ascii=False) if sample else "(no result)"
            except Exception as e:
                preview = f"[Error rendering preview: {e}]"

            results.append(f"""✅ Evaluation completed using model: {model}
📄 Output file: {result_path}

📌 Method: {'🤖 LLM-based' if evaluation_match_source == 'llm' else '🔍 Traditional rule-based'}
📊 Preview of one result:
{preview}
""")
        else:
            results.append(f"❌ Evaluation file not found: {result_path}")

    GLOBAL_STATE_INDEX = build_runtime_state_index(RESULTS, rebuild=True)

    return "\n\n".join(results) if results else "⚠️ No evaluation results available for any model."


# === 6. Fix Suggestions ===
class FixInput(BaseModel):
    fixes_match_source: Optional[str] = Field(
        None, description="Choose 'llm' or 'traditional' to indicate the source of evaluated metadata."
    )

@tool(args_schema=FixInput, return_direct=True)
@track_latency("tool_generate_fixes")
def tool_generate_fixes(fixes_match_source: Optional[str] = None) -> str:
    """
    Generate ESG fix suggestions for incomplete or non-compliant disclosures based on evaluated metadata.

    Match source options:
    - 'llm': Uses AI-evaluated metadata to generate disclosure improvement suggestions.
    - 'traditional': Uses heuristic-based evaluations for generating suggestions.

    If not provided, the tool will prompt for input.
    """
    from pathlib import Path
    import json
    import os

    fix_dir = "fix_outputs"
    meta_dir = "metadata_outputs"
    os.makedirs(fix_dir, exist_ok=True)
    os.makedirs(meta_dir, exist_ok=True)

    global SELECTED_MODELS, RESULTS, GLOBAL_STATE_INDEX
    if not SELECTED_MODELS:
        return "❌ No models selected. Please run `select_pipeline_models()` first."

    if "fixes" not in RESULTS or not isinstance(RESULTS["fixes"], dict):
        RESULTS["fixes"] = {}

    if not os.path.exists("admin/all_disclosures.json"):
        return "❌ Disclosure rules not found in the admin directory."

    if fixes_match_source is None:
        return (
            "❓ Which evaluation result would you like to use to generate fixes?\n\n"
            "**Options:**\n"
            "- `llm`: 🤖 LLM-evaluated metadata\n"
            "- `traditional`: 📘 Rule-based evaluation\n\n"
            "Please provide your choice."
        )

    fixes_match_source = fixes_match_source.lower().strip()
    if fixes_match_source not in ["llm", "traditional"]:
        return "❌ Invalid source. Use either `'llm'` or `'traditional'`."

    with open("admin/all_disclosures.json", "r", encoding="utf-8") as f:
        disclosures = json.load(f)

    rule_vectorstore = build_disclosure_rule_vectorstore(disclosures)
    results = []

    for model in SELECTED_MODELS:
        eval_path = Path(f"{meta_dir}/metadata_eval_{model}_from_{fixes_match_source}.json")
        if not eval_path.exists():
            msg = f"❌ Evaluation results not found for model: {model} at {eval_path}"
            print(msg)
            results.append(msg)
            continue

        with open(eval_path, "r", encoding="utf-8") as f:
            compliance_report = list(json.load(f).values())

        clean_report = []
        for r in compliance_report:
            if isinstance(r, dict):
                r["rating"] = r.get("rating", "❓")
                r["reason"] = r.get("reason") or r.get("justification", "No reason provided")
                r["disclosure_id"] = r.get("disclosure_id") or r.get("id") or "[unknown]"
                clean_report.append(r)

        try:
            raw_fixes = generate_esg_fixes_with_retrieval_multi(
                compliance_report=clean_report,
                rule_vectorstore=rule_vectorstore,
                model_configs=providers,
                selected_models=[model],
                return_raw=True,
                fixes_match_source=fixes_match_source
            )
        except Exception as e:
            error = f"❌ Error generating fixes for model '{model}': {e}"
            print(error)
            results.append(error)
            continue

        output_path = f"{fix_dir}/esg_fixes_{model}_from_{fixes_match_source}.json"
        if Path(output_path).exists():
            try:
                with open(output_path, "r", encoding="utf-8") as f:
                    result_data = json.load(f)
                RESULTS["fixes"][model] = result_data
                sample = result_data[0] if result_data else {}

                try:
                    preview = json.dumps(sample, indent=2, ensure_ascii=False) if sample else "(no fix generated)"
                except Exception as e:
                    preview = f"[Error rendering preview: {e}]"

                source_readable = "🤖 LLM-based" if fixes_match_source == "llm" else "📘 Traditional"
                results.append(f"""✅ Fix suggestions generated using model: {model}
📄 Output file: {output_path}

🔧 Method: {source_readable}
🛠️ Preview of first fix:
{preview}
""")
            except Exception as e:
                results.append(f"❌ Failed to load saved fixes for model {model}: {e}")
        else:
            results.append(f"⚠️ Fix generation completed, but no output file found for model: {model}")

    GLOBAL_STATE_INDEX = build_runtime_state_index(RESULTS, rebuild=True)

    return "\n\n".join(results) if results else "⚠️ No fix suggestions were generated for any model."




# === Utility Tools ===

from pydantic import BaseModel, Field
from typing import Optional

class QueryESGStateInput(BaseModel):
    question: str = Field(..., description="The user’s natural language query about ESG pipeline outputs.")
    section: Optional[str] = Field(None, description="One of: matching, metadata, evaluation, fixes")
    model: Optional[str] = Field(None, description="Optional model filter: gpt, claude, or gemini")
    top_k: int = Field(30, description="Number of results to retrieve (default: 30)")


@tool(args_schema=QueryESGStateInput, return_direct=True)
def query_esg_state(question: str, section: Optional[str] = None, model: Optional[str] = None, top_k: int = 30) -> str:
    """Semantic query over ESG pipeline outputs (matched disclosures, metadata, evaluations, fixes)."""
    import re
    global GLOBAL_STATE_INDEX
    if not GLOBAL_STATE_INDEX:
        return "❌ ESG state index is not initialized. Please run earlier steps to populate outputs first."

    # === Heuristic Section Inference ===
    if not section:
        lowered = question.lower()
        if "match" in lowered or "alignment" in lowered:
            section = "matching"
        elif "metadata" in lowered or "extract" in lowered:
            section = "metadata"
        elif "evaluation" in lowered or "compliance" in lowered:
            section = "evaluation"
        elif "fix" in lowered or "improve" in lowered:
            section = "fixes"

    # === Normalize Model Name ===
    model_alias_map = {
        "gpt": "gpt", "gpt4": "gpt", "gpt-4": "gpt",
        "claude": "claude", "opus": "claude",
        "gemini": "gemini", "gemini-pro": "gemini"
    }
    normalized_model = model_alias_map.get(model.lower(), model.lower()) if model else None

    metadata_filter = {}
    if section:
        metadata_filter["section"] = section
    if normalized_model:
        metadata_filter["model"] = normalized_model

    retriever = GLOBAL_STATE_INDEX.as_retriever(
        search_kwargs={"k": max(top_k, 5)},
        filter=metadata_filter if metadata_filter else None
    )

    results = retriever.invoke(question)
    if not results:
        tip = "You may need to run earlier tools (e.g., matching, metadata extraction) before querying this section."
        return f"⚠️ No relevant results found for your query.\n💡 Tip: {tip}"

    response_lines = []
    for idx, doc in enumerate(results, 1):
        sec = doc.metadata.get("section", "")
        disclosure_id = doc.metadata.get("id", "")
        rating = doc.metadata.get("rating", "")
        reason = doc.metadata.get("reason", "")
        model_name = doc.metadata.get("model", "unknown").upper()
        content = doc.page_content.strip()

        if sec == "matching":
            try:
                parsed = json.loads(content)
                if parsed.get("status") != "matched":
                    continue
                formatted = json.dumps(parsed, indent=2, ensure_ascii=False)
                response_lines.append(
                    f"📌 Match {idx} — Section: {sec.upper()} | Model: {model_name}\n"
                    f"```json\n{formatted}\n```"
                )
            except Exception:
                continue
        else:
            preview = content[:300] + ("..." if len(content) > 300 else "")
            response_lines.append(
                f"📌 Result {idx} — Section: {sec.upper()} | Model: {model_name}\n"
                f"📄 Disclosure: {disclosure_id or '[unknown]'}\n"
                f"📊 Rating: {rating or '[n/a]'}\n"
                f"📝 Reason: {reason or '[none]'}\n"
                f"🔍 Content Preview: {preview}"
            )

    summary_note = (
        f"<thinking>Query retrieved {len(response_lines)} result(s) "
        f"from section='{section or 'any'}', model='{model or 'any'}'.</thinking>"
    )

    return summary_note + "\n\n" + "\n\n".join(response_lines)
@tool(args_schema=QueryESGStateInput, return_direct=True)
def query_esg_report(question: str, top_k: int = 20) -> str:
    """Semantic query over the full content of the uploaded ESG report."""
    global ESG_VECTORSTORE
    if not ESG_VECTORSTORE:
        return "❌ ESG content index is not initialized. Please upload and extract an ESG report first."

    if not question.strip():
        return "⚠️ Your query appears empty. Please enter a meaningful question about the ESG content."

    # Run semantic retrieval
    docs = ESG_VECTORSTORE.similarity_search(question, k=top_k)
    if not docs:
        return "⚠️ No relevant content found for your query within the report."

    response_lines = []
    for i, doc in enumerate(docs, 1):
        content = doc.page_content.strip()
        preview = content[:400] + ("..." if len(content) > 400 else "")
        chunk_id = doc.metadata.get("chunk_index", f"{i - 1}")

        response_lines.append(
            f"🔹 Match {i} — Chunk ID: {chunk_id}\n"
            f"```text\n{preview}\n```"
        )

    return (
        f"<thinking>Top {len(response_lines)} relevant chunk(s) retrieved from the ESG report.</thinking>\n\n"
        + "\n\n".join(response_lines)
    )


# Orchestration Agent with token usage tracker

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from langchain.callbacks import get_openai_callback
import time  # ⏱️ For measuring latency

# === Ensure Global Latency Field Exists ===
if "total_latency_sec" not in LLM_COST_LOG:
    LLM_COST_LOG["total_latency_sec"] = 0.0

# === Display Helper ===
def display_token_usage(current, total):
    html = f"""
    <div style="font-family:Segoe UI, sans-serif; font-size:15px; margin-top:20px;">
        <div style="background:#f8f8f8; border-left:4px solid #4CAF50; padding:12px; border-radius:8px;">
            <b>🧾 Token Usage:</b><br>
            <span style="color:#333;">Prompt Tokens:</span> {current['prompt_tokens']}<br>
            <span style="color:#333;">Completion Tokens:</span> {current['completion_tokens']}<br>
            <span style="color:#333;">Total Tokens:</span> {current['total_tokens']}<br>
            <span style="color:#333;">Cost:</span> ${current['total_cost']:.6f}<br>
            <span style="color:#333;">⏱️ Response Time:</span> {current['latency_sec']:.2f} seconds
        </div>
        <div style="margin-top:12px; background:#f1f1f1; border-left:4px solid #2196F3; padding:12px; border-radius:8px;">
            <b>📊 Cumulative Total:</b><br>
            <span style="color:#333;">Prompt Tokens:</span> {total['prompt_tokens']}<br>
            <span style="color:#333;">Completion Tokens:</span> {total['completion_tokens']}<br>
            <span style="color:#333;">Total Tokens:</span> {total['total_tokens']}<br>
            <span style="color:#333;">Total Cost:</span> ${total['total_cost']:.6f}<br>
            <span style="color:#333;">⏱️ Total Latency:</span> {total['total_latency_sec']:.2f} seconds
        </div>
    </div>
    """
    display(HTML(html))

# === Main Follow-up Chat Widget ===# === Main Follow-up Chat Widget ===
def enable_followup_input(task_name="ESG Report"):
    input_box = widgets.Text(
        placeholder=f"Ask a question about your '{task_name}'...",
        layout=widgets.Layout(width="75%", padding="6px")
    )

    submit_button = widgets.Button(
        description="Submit",
        button_style="primary",
        layout=widgets.Layout(width="80px")
    )

    output_area = widgets.Output(layout=widgets.Layout(
        border='1px solid #ccc', padding="10px", max_height="400px", overflow_y="auto")
    )

    def on_submit(_):
        user_question = input_box.value.strip()
        input_box.value = ""

        if not user_question:
            with output_area:
                clear_output()
                display(HTML("<span style='color:#d33;'>⚠️ Please type a question before submitting.</span>"))
            return

        with output_area:
            clear_output()
            display(HTML(f"""
                <div style="font-family:Segoe UI, sans-serif; margin-bottom:10px;">
                    <b>📨 User Input:</b>
                    <div style="background:#f1f1f1; padding:8px 10px; border-radius:6px; margin-top:5px;">
                        {user_question}
                    </div>
                    <div style="color:#888; margin-top:8px;">⏳ Processing your request...</div>
                </div>
            """))

        try:
            start_time = time.time()
            with get_openai_callback() as cb:
                result = agent.invoke({"input": user_question})
                latency = round(time.time() - start_time, 2)

                usage = {
                    "prompt_tokens": cb.prompt_tokens,
                    "completion_tokens": cb.completion_tokens,
                    "total_tokens": cb.total_tokens,
                    "total_cost": cb.total_cost,
                    "latency_sec": latency
                }

                # ✅ Update cumulative tracker
                LLM_COST_LOG["prompt_tokens"] += cb.prompt_tokens
                LLM_COST_LOG["completion_tokens"] += cb.completion_tokens
                LLM_COST_LOG["total_tokens"] += cb.total_tokens
                LLM_COST_LOG["total_cost"] += cb.total_cost
                LLM_COST_LOG["total_latency_sec"] += latency
                log_entry = {
                    "question": user_question,
                    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "prompt_tokens": cb.prompt_tokens,
                    "completion_tokens": cb.completion_tokens,
                    "total_tokens": cb.total_tokens,
                    "total_cost": round(cb.total_cost, 6),
                    "latency_sec": latency
                }
                with open("/content/agent_logs.jsonl", "a", encoding="utf-8") as f:
                    f.write(json.dumps(log_entry) + "\n")


            output_text = result.get("output", "🤷 No answer found.")
            output_text_html = output_text.replace('\n', '<br/>')

            with output_area:
                clear_output()
                display(HTML(f"""
                    <div style="font-family:Segoe UI, sans-serif;">
                        <b>📨  User Input:</b>
                        <div style="background:#f1f1f1; padding:8px 10px; border-radius:6px; margin-top:5px;">
                            {user_question}
                        </div>

                        <b style="display:block; margin-top:15px;">🧠 Response:</b>
                        <div style="background:#f9f9f9; border-left:4px solid #4CAF50; padding:10px; border-radius:6px; margin-top:5px;">
                            {output_text_html}
                        </div>
                    </div>
                """))
                display_token_usage(usage, LLM_COST_LOG)

        except Exception as e:
            with output_area:
                clear_output()
                display(HTML(f"<div style='color:#c00;'>❌ Error: {e}</div>"))

    submit_button.on_click(on_submit)
    input_box.on_submit(on_submit)  # ✅ ENTER key submits

    display(widgets.VBox([
        widgets.HBox([input_box, submit_button]),
        output_area
    ]))


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
from langchain.memory import ConversationBufferMemory

# === Global Token Tracker ===
LLM_COST_LOG = {
    "prompt_tokens": 0,
    "completion_tokens": 0,
    "total_tokens": 0,
    "total_cost": 0.0,
    "total_latency_sec": 0.0
}

# === Function to Show Cumulative Token Summary ===
def print_agent_token_usage():
    print("🧾 Agent Token Usage Summary:")
    print(f"🔹 Prompt Tokens:     {LLM_COST_LOG['prompt_tokens']}")
    print(f"🔹 Completion Tokens: {LLM_COST_LOG['completion_tokens']}")
    print(f"🔹 Total Tokens:      {LLM_COST_LOG['total_tokens']}")
    print(f"💵 Total Cost (USD):  ${LLM_COST_LOG['total_cost']:.6f}")

# === Create plain LLM ===
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.2)

# === Tool Registry ===
tools = [
    tool_upload_esg_pdf,
    tool_list_framework_options,
    tool_select_framework,
    tool_list_sasb_sectors,
    tool_select_sasb_sectors,
    tool_run_matching,
    tool_extract_metadata,
    tool_evaluate_metadata,
    tool_generate_fixes,
    query_esg_state,
    query_esg_report
]

# === Prompt for Orchestration ===
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Always start with upload tool"),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# === Build Agent ===
agent_fn = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = AgentExecutor.from_agent_and_tools(
    agent=agent_fn,
    tools=tools,
    memory=memory,
    verbose=False,
    return_intermediate_steps=False,
    handle_parsing_errors=True,
    stream_runnable=False
)


In [ ]:
def select_pipeline_models():
    import ipywidgets as widgets
    from IPython.display import display, Markdown, clear_output

    global SELECTED_MODELS
    if SELECTED_MODELS is None:
        SELECTED_MODELS = []

    description = """
### 💡 ESG Pipeline – LLM Model Selection

This ESG pipeline uses language models (LLMs) in the following steps:

1. **Metadata Extraction** – Extract KPIs and narrative content
2. **Compliance Evaluation** – Check if disclosures are satisfied
3. **Fix Generation** – Suggest missing elements or improvements

Please choose one or more LLMs to be used throughout these steps:
"""

    display(Markdown(description))

    options = ["gpt", "claude", "gemini"]
    checkboxes = [
        widgets.Checkbox(value=(opt in SELECTED_MODELS or (not SELECTED_MODELS and opt == "gpt")), description=opt)
        for opt in options
    ]
    submit_button = widgets.Button(description="Confirm Selection", button_style="success")
    output = widgets.Output()
    def on_submit(_):
        selected = [cb.description for cb in checkboxes if cb.value]
        with output:
            clear_output()
            if not selected:
                print("❌ Please select at least one model.")
            else:
                SELECTED_MODELS.clear()
                SELECTED_MODELS.extend(selected)
                print(f"✅ Models selected: {', '.join(SELECTED_MODELS)}")

    submit_button.on_click(on_submit)
    display(widgets.VBox(checkboxes + [submit_button, output]))

select_pipeline_models()

In [ ]:
import time
from langchain.callbacks import get_openai_callback

start_time = time.time()
with get_openai_callback() as cb:
    result = agent.invoke({"input": "Let's begin processing an ESG report."})
    latency = time.time() - start_time

    print(result["output"])

    # Update global tracker
    LLM_COST_LOG["prompt_tokens"] += cb.prompt_tokens
    LLM_COST_LOG["completion_tokens"] += cb.completion_tokens
    LLM_COST_LOG["total_tokens"] += cb.total_tokens
    LLM_COST_LOG["total_cost"] += cb.total_cost
    LLM_COST_LOG["total_latency_sec"] += latency
    log_entry = {
    "question": "Let's begin processing an ESG report.",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "prompt_tokens": cb.prompt_tokens,
    "completion_tokens": cb.completion_tokens,
    "total_tokens": cb.total_tokens,
    "total_cost": round(cb.total_cost, 6),
    "latency_sec": round(latency, 2)
}
with open("/content/agent_logs.jsonl", "a", encoding="utf-8") as f:
    f.write(json.dumps(log_entry) + "\n")


    print("🧾 Initial Agent Token Usage:")
    print(f"🔹 Prompt: {cb.prompt_tokens}, Completion: {cb.completion_tokens}, Total: {cb.total_tokens}")
    print(f"💵 Cost: ${cb.total_cost:.6f}")
    print(f"⏱️ Latency: {latency:.2f} seconds")

# Enable interaction
enable_followup_input("ESG Report")
